In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as ticker

### Figures for ACE-EAMv3 AMIP evaluation

In [ ]:
path = "/pscratch/sd/e/elynnwu/fme-output/greens-experiment-v3/beaker-RS2/amip-1970-2020/"
annual = xr.open_dataset(path+"annual_diagnostics.nc")
time_mean = xr.open_dataset(path+"time_mean_diagnostics.nc")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes = axes.flatten()
i = 0
titles = ["Net TOA radiation", "TOA LW up", "TOA SW up"]
for var in ["net_energy_flux_toa_into_atmosphere", "FLUT", "top_of_atmos_upward_shortwave_flux"]:
    axes[i].plot(annual.year, annual[var].sel(source="prediction", sample=0), label="ACE")
    mean = annual[var].sel(source="prediction", sample=0).mean()
    std = annual[var].sel(source="prediction", sample=0).std()
    print(f"ACE mean: {mean.values}, std: {std.values}")
    axes[i].plot([1971, 2020], [mean, mean], linestyle="--", color="tab:blue")
    axes[i].plot(annual.year, annual[var].sel(source="target", sample=0), label="EAMv3", color="k")
    mean = annual[var].sel(source="target", sample=0).mean()
    std = annual[var].sel(source="target", sample=0).std()
    print(f"E3SM mean: {mean.values}, std: {std.values}")
    axes[i].plot([1971, 2020], [mean, mean], linestyle="--", color="k")

    axes[i].set_title(titles[i])
    axes[i].set_xlim([1971, 2020])
    axes[i].yaxis.set_major_locator(ticker.MultipleLocator(0.5))
    i += 1

axes[0].legend()
axes[0].set_ylabel(r"[W/m$^2$]")
plt.tight_layout()
fig.patch.set_alpha(0.0)
fig.savefig(
    "figures/ace-eamv3-beaker-RS2-amip-1970-2020.png",
    dpi=200,
    bbox_inches="tight",
)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
axes = axes.flatten()
lons, lats = np.meshgrid(time_mean["lon"], time_mean["lat"])
i = 0
for patch in ["net_energy_flux_toa_into_atmosphere", "FLUT", "top_of_atmos_upward_shortwave_flux"]:
    data = time_mean[f"bias_map-{patch}"]
    cf = axes[i].pcolormesh(lons, lats, data, transform=ccrs.PlateCarree(), cmap="RdBu_r", vmin=-10, vmax=10)
    axes[i].coastlines()
    axes[i].set_global()
    axes[i].set_title(titles[i])
    gl = axes[i].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    i += 1
cbar = fig.colorbar(cf, ax=axes, orientation="vertical", pad=0.02, fraction=0.007)
cbar.set_label(f"[W/m2]")
fig.patch.set_alpha(0.0)

fig.savefig(
    "figures/2025-05-08-ace-eamv3-beaker-RS2-amip-1970-2020-rad-map.png",
    dpi=200,  
    bbox_inches="tight",
)